In [2]:
import pandas as pd
import numpy as np

### Loading and split data, create our own MSE:

In [9]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv(
    './ml-100k/u.data', sep='\t', names=names)

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

# Create r_{ui}, our ratings matrix
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

# Split into training and test sets. 
# Remove 10 ratings for each user 
# and assign them to the test set
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

train, test = train_test_split(ratings)
indicating_mat = np.vectorize(lambda x: 0 if x==0 else 1)(train)
mask = indicating_mat == 1

from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [3]:
n_user, n_item = train.shape

# Hyperparameter:
k = 40
steps = 400
lrate = 0.001
_lambda = 0.001

# Parameters:
b_u = np.zeros(n_user)
b_i = np.zeros(n_item)

#q_i
q_i = np.random.normal(scale=1./k, size=(n_item, k))
#P_u
x_j = np.random.normal(scale=1./k, size=(n_item, k))
#y_i
y_j = np.random.normal(scale=1./k, size=(n_item, k))

global_bias = np.mean(train[np.where(train != 0)])

non_zeros = train.nonzero()

# This is equivalent of taking the length and doing the square root.
# N = np.power(indicating_mat.sum(1), -0.5)
N = 1./np.linalg.norm(indicating_mat, axis = 1)
R = 1./np.linalg.norm(train, axis = 1)


def get_item_item (u, mu, b_i, b_u, x_j):
    temp = np.zeros(k)
    for j in indicating_mat[u,:].nonzero()[0]:
        temp += (ratings[u, j] - mu - b_u[u] - b_i[j])*x_j[j, :]
    return temp * R[u]
        
def predict_one(u, i, b_i, b_u, q_i, x_j, y_j):
    item_item = get_item_item(u, global_bias, b_i, b_u, x_j)
    return global_bias + b_i[i] + b_u[u] + \
    q_i[i,:].T.dot(item_item + N[u] * y_j[mask[u,:], :].sum(axis=0))

def svdasy_step():
    rows = np.random.permutation(len(non_zeros[0]))
    for i in rows:
        user = non_zeros[0][i]
        item = non_zeros[1][i]
        pred = predict_one(user, item, b_i, b_u, q_i, x_j, y_j)

        ## Watch out to turn learning rate separately, this needs to be calculate separately
        error = train[user][item] - pred
        
        b_u[user] += lrate*(error - _lambda * b_u[user])
        b_i[item] += lrate*(error - _lambda * b_i[item])
        
        
        item_item = get_item_item(user, global_bias, b_i, b_u, x_j)
        ## Update for q_i (item vector)
        q_i[item, :] += lrate * (error * (item_item + N[user]* y_j[mask[user, :], :].sum(axis=0)) -\
                                  _lambda * q_i[item, :])
        
        # Update for x_j (user vector)
        j = indicating_mat[user,:].nonzero()
        x_j[j, :] += lrate * (error * item_item * q_i[item, :] - _lambda * x_j[j, :])
        
        # Update for each y_j 
        temp = error * N[user] * q_i[item, :]
        j = indicating_mat[user,:].nonzero()
        y_j[j,:] += lrate * (temp - _lambda * y_j[j,:])

In [4]:
# (ratings[1, 1]- global_bias - b_u[0] - b_i[0]) * x_j[0, :] + np.zeros(k)

In [24]:
data = []
def predict():
    predictions = np.zeros([n_user, n_item])
    for user in range(n_users):
        for item in range(n_items):
            predictions[user, item] = predict_one(user, item, b_i, \
                        b_u, q_i, x_j, y_j)
    data.append([get_mse(predictions, train),get_mse(predictions, test)])
    with open('./asym.csv', 'a') as f:
        f.write(get_mse(predictions, train)+','+get_mse(predictions, test)+' \n')
    


In [5]:
for i in range(steps):
    print(i)
    svdasy_step()
    if i%10 == 0:
        predict()

0
1
2
3
4
5
6
7
8
9


In [10]:

dtf = pd.DataFrame(data)
dtf.to_csv("asymResult.csv")

In [16]:
# get_mse(predictions, test)

In [15]:
# get_mse(predictions, train)

In [14]:
# pd.read_csv('./asymResult.csv')